#### Fase 1. Importación de datos en Spark

*Fuente: Cory Maklin */machine-learning-at-scale-with-apache-spark-mllib-python-example-b32a9c74c610*

In [0]:

import pandas as pd
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
schema = """`age` DOUBLE,
`workclass` STRING,
`fnlwgt` DOUBLE,
`education` STRING,
`education_num` DOUBLE,
`marital_status` STRING,
`occupation` STRING,
`relationship` STRING,
`race` STRING,
`sex` STRING,
`capital_gain` DOUBLE,
`capital_loss` DOUBLE,
`hours_per_week` DOUBLE,
`native_country` STRING,
`income` STRING"""

df = spark.read.csv("/databricks-datasets/adult/adult.data", header=True, schema=schema)

In [0]:
df.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
 age| workclass| fnlwgt| education|education_num| marital_status| occupation| relationship| race| sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
50.0| Self-emp-not-inc| 83311.0| Bachelors| 13.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 0.0| 0.0| 13.0| United-States| <=50K|
38.0| Private|215646.0| HS-grad| 9.0| Divorced| Handlers-cleaners| Not-in-family| White| Male| 0.0| 0.0| 40.0| United-States| <=50K|
53.0| Private|234721.0| 11th| 7.0| Married-civ-spouse| Handlers-cleaners| Husband| Black| Male| 0.0| 0.0| 40.0| United-States| <=50K|
28.0| Private|338409.0| Bachelors| 13.0| Married-civ-spouse| Prof-specialty| Wife| Black| Female| 0.0| 0.0| 40.0| Cuba| <=50K|
37.0| Private|284582.0| Masters| 14.0| Married-civ-spouse| Exec-managerial| Wife| White| Female| 0.0| 0.0| 40.0| United-States| <=50K|
49.0| Private|160187.0| 9th| 5.0| Married-spouse-a...| Other-service| Not-in-family| Black| Female| 0.0| 0.0| 16.0| Jamaica| <=50K|
52.0| Self-emp-not-inc|209642.0| HS-grad| 9.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 0.0| 0.0| 45.0| United-States| >50K|
31.0| Private| 45781.0| Masters| 14.0| Never-married| Prof-specialty| Not-in-family| White| Female| 14084.0| 0.0| 50.0| United-States| >50K|
42.0| Private|159449.0| Bachelors| 13.0| Married-civ-spouse| Exec-managerial| Husband| White| Male| 5178.0| 0.0| 40.0| United-States| >50K|
37.0| Private|280464.0| Some-college| 10.0| Married-civ-spouse| Exec-managerial| Husband| Black| Male| 0.0| 0.0| 80.0| United-States| >50K|
30.0| State-gov|141297.0| Bachelors| 13.0| Married-civ-spouse| Prof-specialty| Husband| Asian-Pac-Islander| Male| 0.0| 0.0| 40.0| India| >50K|
23.0| Private|122272.0| Bachelors| 13.0| Never-married| Adm-clerical| Own-child| White| Female| 0.0| 0.0| 30.0| United-States| <=50K|
32.0| Private|205019.0| Assoc-acdm| 12.0| Never-married| Sales| Not-in-family| Black| Male| 0.0| 0.0| 50.0| United-States| <=50K|
40.0| Private|121772.0| Assoc-voc| 11.0| Married-civ-spouse| Craft-repair| Husband| Asian-Pac-Islander| Male| 0.0| 0.0| 40.0| ?| >50K|
34.0| Private|245487.0| 7th-8th| 4.0| Married-civ-spouse| Transport-moving| Husband| Amer-Indian-Eskimo| Male| 0.0| 0.0| 45.0| Mexico| <=50K|
25.0| Self-emp-not-inc|176756.0| HS-grad| 9.0| Never-married| Farming-fishing| Own-child| White| Male| 0.0| 0.0| 35.0| United-States| <=50K|
32.0| Private|186824.0| HS-grad| 9.0| Never-married| Machine-op-inspct| Unmarried| White| Male| 0.0| 0.0| 40.0| United-States| <=50K|
38.0| Private| 28887.0| 11th| 7.0| Married-civ-spouse| Sales| Husband| White| Male| 0.0| 0.0| 50.0| United-States| <=50K|
43.0| Self-emp-not-inc|292175.0| Masters| 14.0| Divorced| Exec-managerial| Unmarried| White| Female| 0.0| 0.0| 45.0| United-States| >50K|
40.0| Private|193524.0| Doctorate| 16.0| Married-civ-spouse| Prof-specialty| Husband| White| Male| 0.0| 0.0| 60.0| United-States| >50K|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
only showing top 20 rows

In [0]:
df.toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
1,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
2,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
3,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
4,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K


In [0]:
df.dtypes

Out[54]: [('age', 'double'),
 ('workclass', 'string'),
 ('fnlwgt', 'double'),
 ('education', 'string'),
 ('education_num', 'double'),
 ('marital_status', 'string'),
 ('occupation', 'string'),
 ('relationship', 'string'),
 ('race', 'string'),
 ('sex', 'string'),
 ('capital_gain', 'double'),
 ('capital_loss', 'double'),
 ('hours_per_week', 'double'),
 ('native_country', 'string'),
 ('income', 'string')]

In [0]:
df.count()

Out[55]: 32560

#### Fase 2. Pre-procesamiento de datos

In [0]:
categorical_variables = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]
encoder = OneHotEncoder(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)
assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

In [0]:
pipeline = Pipeline(stages=indexers + [encoder, assembler])

In [0]:
df = pipeline.fit(df).transform(df)

In [0]:
df.printSchema()

root
-- age: double (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: double (nullable = true)
-- education: string (nullable = true)
-- education_num: double (nullable = true)
-- marital_status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital_gain: double (nullable = true)
-- capital_loss: double (nullable = true)
-- hours_per_week: double (nullable = true)
-- native_country: string (nullable = true)
-- income: string (nullable = true)
-- workclass-index: double (nullable = false)
-- education-index: double (nullable = false)
-- marital_status-index: double (nullable = false)
-- occupation-index: double (nullable = false)
-- relationship-index: double (nullable = false)
-- race-index: double (nullable = false)
-- sex-index: double (nullable = false)
-- native_country-index: double (nullable = false)
-- workclass-index-encoded: vector (nullable = true)
-- occupation-index-encoded: vector (nullable = true)
-- marital_status-index-encoded: vector (nullable = true)
-- education-index-encoded: vector (nullable = true)
-- native_country-index-encoded: vector (nullable = true)
-- race-index-encoded: vector (nullable = true)
-- relationship-index-encoded: vector (nullable = true)
-- sex-index-encoded: vector (nullable = true)
-- categorical-features: vector (nullable = true)

In [0]:
continuous_variables = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
assembler = VectorAssembler(
    inputCols=['categorical-features', *continuous_variables],
    outputCol='features'
)

In [0]:
df = assembler.transform(df)

In [0]:
indexer = StringIndexer(inputCol='income', outputCol='label')
df = indexer.fit(df).transform(df)
df.limit(10).toPandas()['label']

/databricks/spark/python/pyspark/sql/pandas/conversion.py:93: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
 Unable to convert the field workclass-index-encoded. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Context: Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
 warnings.warn(msg)
Out[62]: 0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 1.0
7 1.0
8 1.0
9 1.0
Name: label, dtype: float64

#### Fase 3. Entrenamiento del modelo

In [0]:
(train_df, test_df) = df.randomSplit([0.8,0.2])

In [0]:
lr = LogisticRegression(featuresCol='features', labelCol='label')
model = lr.fit(train_df)

#### Fase 4. Predicciones y evaluación

In [0]:
pred = model.transform(test_df)
pred.limit(10).toPandas()[['label', 'prediction']]

,label,prediction
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0
